# Virtual Potential Fields

This is adapted from the given intro_coppelia file from near the beginning of Lehigh University's Mobile Robotics course.

In [ ]:
# Import the python library that connects to CoppeliaSim, the file is sim.py.
try:
    import sim
except:
    print('--------------------------------------------------------------')
    print('"sim.py" could not be imported. This means very probably that')
    print('either "sim.py" or the remoteApi library could not be found.')
    print('Make sure both are in the same folder as this file,')
    print('or appropriately adjust the file "sim.py"')
    print('--------------------------------------------------------------')
    print('')

import time
import numpy as np
from numbers import Number

%pylab inline

### Check if python is connecting to Coppelia

In [ ]:
print('Program started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    sim.simxGetPingTime(clientID)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
else:
    print('Failed connecting to remote API server')
print('Program ended')

# Object oriented programming

In [ ]:
class robot():
    
    def __init__(self, motor_names, frame_name=None):        
        self.client_id = self.open_connection()
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame = self._get_handler(frame_name)
        
        #Dummy
        self.dummy_handle = None
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object):
        relative_handle = self._get_handler(relative_object)
        sim.simxSetObjectPosition(clientID, self.frame, relative_handle, position, sim.simx_opmode_oneshot)
    
    def get_position(self, relative_object=-1):
        if relative_object != -1:
            relative_object = self.get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_position(self, object_name):
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)
    
    def add_dummy(self, position):
        #add dummy if does not exist
        res, self.dummy_handle = sim.simxCreateDummy(self.client_id, 1, None, sim.simx_opmode_blocking)
        
        #move dummy to position
        sim.simxSetObjectPosition(clientID, self.dummy_handle, -1, position, sim.simx_opmode_oneshot)
    
    def move_dummy(self, position):
        #add dummy if does not exist
        if self.dummy_handle is None:
            res, self.dummy_handle = sim.simxCreateDummy(self.client_id, 1, None, sim.simx_opmode_blocking)
        
        #move dummy to position
        sim.simxSetObjectPosition(clientID, self.dummy_handle, -1, position, sim.simx_opmode_oneshot)
        
    def get_dummy_relative_position(self):
        position = [0, 0, 0]
        if self.dummy_handle is not None:
            res, position = sim.simxGetObjectPosition(self.client_id, self.dummy_handle, self.frame, sim.simx_opmode_blocking)
        return array(position)
    
    #For cuboid plotting: Get minimum and maximum x and y bounding box coords
    #using simxGetObjectIntParameter(clientID, objectHandle, parameterID, sim.simx_opmode_blocking)
    #where parameterID is 15 for minX, 18 for maxX, 16 for minY, and 19 for maxY.
    #Subtract minY from maxY to get height, subtract minX from maxX to get width.
    #Add global object origin position to minX and minY to get absolute positions of that corner.
    #Get rotation relative to world frame about Z axis.
    #Return minX, minY, width, height, rotation.
    def get_cuboid_plotting_data(self, object_name):
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        #global position
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        position.pop()
        #get rotation
        res, rotation=sim.simxGetObjectOrientation(clientID, object_h, -1, sim.simx_opmode_blocking)
        rotation = rotation[2]
        
        return rotation, position
    
    def get_cylinder_plotting_data(self, object_name):
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        #global position
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        position.pop()
        #radius
        res, radius = sim.simxGetObjectFloatParameter(clientID, object_h, 18, sim.simx_opmode_blocking)
        
        return position, radius

In [ ]:
#Rotation function from stackoverflow user Mark Dickenson at this link: 
#https://stackoverflow.com/questions/34372480/rotate-point-about-another-point-in-degrees-python
def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def get_cuboid_obstacle(cuboidName,r):
    rotation, center = r.get_cuboid_plotting_data(cuboidName)
    circles = []
    obstacle = []
    circles.append([center[0], center[1]])
    circles.append([center[0] - 1.75, center[1]])
    circles.append([center[0] - 1.5, center[1]])
    circles.append([center[0] - 1, center[1]])
    circles.append([center[0] - .5, center[1]])
    circles.append([center[0] + .5, center[1]])
    circles.append([center[0] + 1, center[1]])
    circles.append([center[0] + 1.5, center[1]])
    circles.append([center[0] + 1.75, center[1]])
    #rotate circles by rotation
    for circle in circles:
        x, y = rotate(center, circle, rotation)
        obstacle.append([x, y])
    
    return obstacle

def get_cylinder_obstacle(cylinderName,r, obsK):
    position, radius = r.get_cylinder_plotting_data(cylinderName)
    cylinders = []
    cylinders.append([position[0], position[1], obsK])
    cylinders.append([position[0] + .5, position[1], obsK])
    cylinders.append([position[0], position[1] + .5, obsK])
    cylinders.append([position[0] - .5, position[1], obsK])
    cylinders.append([position[0], position[1] -.5, obsK])

    return cylinders

# Construct Map of Scene

Now, we can construct a map of the scene using the robot class, as long as we know what all the scene obstacles are named.

In [ ]:
#First, we get the obstacle and goal coordinates

# Joint names in coppelia for KUKA omnidirectional robot
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']

r = robot(motor_names, 'Omnirob')  # Create an instance of our robot

#Set scaling factors for goal potential and obstacle potential
obsK = .4
goalK = .1

#get robot position
robotXY = r.get_position().tolist()
robotXY.pop()

#get goal position
goal = r.get_object_position('Sphere9').tolist()
goal.pop()
#append goal K value
goal.append(goalK)

#get obstacle positions
obstacles = []

obstacles += get_cylinder_obstacle("Cylinder6", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder7", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder8", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder9", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder10", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder11", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder12", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder13", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder14", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder15", r, obsK)
obstacles += get_cylinder_obstacle("Cylinder16", r, obsK)
#append obstacle K value
obstaclesK = np.zeros((len(obstacles), 1))
obstaclesK.fill(obsK)
obstacles = np.hstack((np.asarray(obstacles), obstaclesK)).tolist()

r.close_connection()

In [ ]:
#Next, we plot the map without the vector field

from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

def get_obstacle_patches(obstacles, radius):
    obstacleList = []
    for obstacle in obstacles:
        obstacleList.append(Circle((obstacle[0], obstacle[1]), radius))
        
    obstaclePatches = PatchCollection(obstacleList, facecolor='k')
        
    return obstaclePatches

In [ ]:
preview_figure = plt.figure(figsize(15,15))
preview_show = preview_figure.add_subplot(111)

preview_show.add_collection(get_obstacle_patches(obstacles, 0.5))
plt.plot(robotXY[0], robotXY[1], 'bo', goal[0], goal[1], 'go')

plt.grid()
preview_show.set_aspect('equal', adjustable='box')

# Construct Vector Field

Now that we have our map, we can add a vector field for our virtual potential field

In [ ]:
# Functions for calculating vector field

def get_goal_force(goal, position):
    xForceGoal = goalK * (goal[0] - position[0])
    yForceGoal = goalK * (goal[1] - position[1])
    force = np.array([xForceGoal, yForceGoal])
    
    return force

def get_obstacle_force(obstacle, position):
    #Get distances between current position and obstacle
    xDistance = position[0] - obstacle[0]
    yDistance = position[1] - obstacle[1]

    obstacleDistance = sqrt(xDistance**2 + yDistance**2)

    #Use those distances and k-values to compute the force

    #Derivative of potential function:
    potentialDeriv = (obstacle[2] / (obstacleDistance**3))

    #Direction of force (partial derivatives of distance function)
    xForceDir = (xDistance / obstacleDistance)
    yForceDir = (yDistance / obstacleDistance)

    #Forces
    xForce = potentialDeriv * xForceDir
    yForce = potentialDeriv * yForceDir
    
    return [xForce, yForce]

def get_extra_force(virtualObs, position):
    kappaExt = 1.4
    
    xForce = kappaExt * ((position[0] - virtualObs[0]) / abs(position[0] - virtualObs[0]))
    yForce = kappaExt * ((position[1] - virtualObs[1]) / abs(position[1] - virtualObs[1]))
    
    return [xForce, yForce]

# get the cumulative force incorporating each obstacle and goal given a certain position
def get_force_vectors(obstacleList, goal, position, vObs=None):
    #Calculate goal force
    force = get_goal_force(goal, position)
    
    #Calculate obstacle forces, adding goal force each time
    for obstacle in obstacles:
        obsForce = get_obstacle_force(obstacle, position)
        
        force[0] = force[0] + obsForce[0]
        force[1] = force[1] + obsForce[1]
        
    #factor in fake obstacle if necessary
    if vObs is not None:
        extForce = get_extra_force(vObs, position)
        
        force[0] = force[0] + extForce[0]
        force[1] = force[1] + extForce[1]
        
        
    return force[0], force[1]

In [ ]:
#This section adapted from Dr. David Saldana's code at the following link:
#https://github.com/dsaldana/CSE360-MobileRobotics/blob/master/Potential_functions.ipynb

#Grid of visualized vectors
X, Y = np.meshgrid(np.arange(-10, 10, .5), np.arange(-10, 10, .5))
#Get forces for visualized vector points
xForce, yForce = get_force_vectors(obstacles, goal, [X, Y])

#Show forces
preview_figure = plt.figure(figsize(15,15))
preview_show = preview_figure.add_subplot(111)

#preview_show.add_collection(get_obstacle_patches(obstacles, 0.5))
plt.plot(robotXY[0], robotXY[1], 'bo', goal[0], goal[1], 'go')

#Eliminate any vectors that are beyond a certain length
vectorLength = np.hypot(xForce, yForce)
xForce[vectorLength > 5], yForce[vectorLength > 5] = None, None

#Plot vector field
plt.quiver(X, Y, xForce, yForce)

plt.grid()
preview_show.set_aspect('equal', adjustable='box')

In [ ]:
#Set of corners, which act as relative positions for the robot to use
yMin = -1
yMax = 1

corners = np.array([[0, yMin],
                    [0, yMax],
                    [yMin, 0],
                    [yMax, 0],
                    [yMin, yMin],
                    [yMax, yMin],
                    [yMin, yMax],
                    [yMax, yMax]])

# Get robot corner positions
def getRobotBoundingBox(r):
    global corners
    global lastUsed

    roboPos = r.get_position()
    obsPos = corners + [roboPos[0], roboPos[1]]
    
    #Get which obstacle position will have the greatest impact
    index = 0
    maxScale = 0
    maxIndex = 0
    #for each candidate position, compare it to the prior and record the index
    #of the current largest. At the end, you'll have the biggest in maxIndex.
    for pos in obsPos:
        forceX, forceY = get_force_vectors(obstacles, goal, pos)
        if sqrt(forceX**2 + forceY**2) > maxScale:
            maxScale = sqrt(forceX**2 + forceY**2)
            maxIndex = index

        index += 1
    
    corners = np.asarray(corners)
    return obsPos[maxIndex]

# Guide Robot to Destination

Now that we have the virtual potential field, we can have the robot follow it to the goal.

In [ ]:
# Robot movement function. Basic motor velocities for moving in the
# positive x axis and the positive y axis, can scale them and then combine them
# to move in any direction.

def move(xScale, yScale, r):
    motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
    #basic x-axis movement
    xMotor = np.array([-1., -1., 1., 1.])
    yMotor = np.array([-1., 1., 1., -1.])
    
    r.send_motor_velocities(xScale * xMotor + yScale * yMotor)

r = robot(motor_names, 'Omnirob')  # Create an instance of our robot
time.sleep(1) #wait for a bit so that everything syncs properly

timeStep = 0

lastMin = 0
timeSet = False

virtualObs = None
oldGoalDistance = 0
goalDistance = 1000000000
timerPassed = False
while(goalDistance > 0.25):
    #calculate next direction
    roboPos = r.get_position()
    x, y = get_force_vectors(obstacles, goal, [roboPos[0], roboPos[1]], virtualObs)
    
    #convert from global vector to robot frame vector
    globalForce = np.array([x, y]).T

    err, direction = sim.simxGetObjectOrientation(clientID,r.frame, -1,  sim.simx_opmode_blocking)
    rotMatrix = np.array([[cos(direction[2]), -1*sin(direction[2])],
                          [sin(direction[2]), cos(direction[2])]])
    roboForce = rotMatrix.dot(globalForce)
    roboForce = np.array([x,y])
    
    move(roboForce[0], roboForce[1], r)
    time.sleep(0.01)
    ###############################################################################################
    #If robot did not move a significant distance, generate obstacle
    #Get travel distance
    newRoboPos = r.get_position()
    xDistance = newRoboPos[0] - roboPos[0]
    yDistance = newRoboPos[1] - roboPos[1]
    travelDistance = sqrt(xDistance**2 + yDistance**2)
    
    #Get distance towards goal
    xDistanceGoal = newRoboPos[0] - goal[0]
    yDistanceGoal = newRoboPos[1] - goal[1]
    oldGoalDistance = goalDistance
    goalDistance = sqrt(xDistanceGoal**2 + yDistanceGoal**2)
    #If you've moved closer to the goal (or the distance hasn't changed)
    #Then you've escaped the local minimum
    if goalDistance <= oldGoalDistance:
        virtualObs = None
    
    #Check if we need a virtual obstacle, but only after the first timestep
    if travelDistance < 0.001 and timerPassed:
        #Get candidate positions for virtual obstacle (center of robot, or any edge or corner)
        obsPos = getRobotBoundingBox(r)
        virtualObs = obsPos
        r.move_dummy(obsPos)
        
    timerPassed = True

move(0, 0, r)

r.close_connection()